In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision.models import vgg16
import copy
import tqdm
import os
import matplotlib
import pandas as pd
from timeit import default_timer as timer
import copy

In [2]:
transform_train = transforms.Compose(
    [transforms.Pad(4),
     transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
     transforms.RandomHorizontalFlip(),
     transforms.RandomGrayscale(),
     transforms.RandomCrop(32, padding=4),
])
 
transform_test = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]
)
 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
 
trainset = torchvision.datasets.CIFAR10(root='/data', train=True, download=True, transform=transform_train)
trainLoader = torch.utils.data.DataLoader(trainset, batch_size=24, shuffle=True)
 
testset = torchvision.datasets.CIFAR10(root='/data', train=False, download=False, transform=transform_test)
testLoader = torch.utils.data.DataLoader(testset, batch_size=24, shuffle=False)

In [3]:
num_epochs = 40
total = 0
accuracy_rate = []
 
def test():
    model.eval()
    correct = 0  # 预测正确的图片数
    total = 0  # total number of pictures
    with torch.no_grad():
        for data in testLoader:
            images, labels = data
            images = images.to(device)
            outputs = model(images).to(device)
            outputs = outputs.cpu()
            outputarr = outputs.numpy()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
    accuracy = 100 * correct / total
    accuracy_rate.append(accuracy)
    print(f'Accuracy is:{accuracy}%'.format(accuracy))

In [4]:
model = vgg16(pretrained=True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 110MB/s]


# The training of the model
Since we need to load it, here we ignore the code below

In [5]:
# # Train
# # model.load_state_dict(torch.load('CIFAR-model/VGG16.pth'))
# optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=5e-3)
# loss_func = nn.CrossEntropyLoss()
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.4, last_epoch=-1)

# for epoch in range(num_epochs):
#     model.train()
#     model.to(device)
#     running_loss = 0.0
#     total_correct = 0
#     total_trainset = 0
 
#     for i, (data, labels) in enumerate(trainLoader, 0):
#         data = data.to(device)
#         outputs = model(data).to(device)
#         labels = labels.to(device)
#         loss = loss_func(outputs, labels).to(device)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#         _, pred = outputs.max(1)
#         correct = (pred == labels).sum().item()
#         total_correct += correct
#         total_trainset += data.shape[0]
#         if i % 1000 == 0 and i > 0:
#             print(f"{i}th Training, running_loss={running_loss}".format(i, running_loss))
#             running_loss = 0.0
#     test()
#     scheduler.step()

# Load the model of vgg

In [6]:
state_dict = torch.load('./vgg16_2.pth',map_location=torch.device('cpu'))
print(state_dict.keys())
model.load_state_dict(state_dict)

odict_keys(['features.0.weight', 'features.0.bias', 'features.2.weight', 'features.2.bias', 'features.5.weight', 'features.5.bias', 'features.7.weight', 'features.7.bias', 'features.10.weight', 'features.10.bias', 'features.12.weight', 'features.12.bias', 'features.14.weight', 'features.14.bias', 'features.17.weight', 'features.17.bias', 'features.19.weight', 'features.19.bias', 'features.21.weight', 'features.21.bias', 'features.24.weight', 'features.24.bias', 'features.26.weight', 'features.26.bias', 'features.28.weight', 'features.28.bias', 'classifier.0.weight', 'classifier.0.bias', 'classifier.3.weight', 'classifier.3.bias', 'classifier.6.weight', 'classifier.6.bias'])


<All keys matched successfully>

In [7]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# We make a testing for the load model

In [8]:
# def get_gpu_inference_time_and_test_acc(model, test_loader):
#     test_data_size = len(testLoader.dataset)
#     model = model.cuda()
#     # Wam-up
#     # Because I sequentially run each cell above, and they includes for loop, I assume the CPU is warmed up
#     torch.cuda.synchronize()
#     elapsed_time_ms = 0
#     start = timer()
#     # Start Inference Phase
#     correct = 0
#     with torch.no_grad():
#         for imgs, labels in tqdm.tqdm(test_loader):
#             imgs, labels = imgs.cuda(), labels.cuda()
#             # imgs, labels = imgs, labels
#             out = model(imgs)
#             predictions = out.argmax(dim=1, keepdim=True)  
#             correct += predictions.eq(labels.view_as(predictions)).sum().item()
#     accuracy = correct/len(test_loader.dataset)
#     print(accuracy)
#     # End of inference Phase
#     torch.cuda.synchronize()
#     end = timer()
#     elapsed_time_ms = (end - start) * 1000
#     per_sample = elapsed_time_ms/test_data_size
#     print("GPU Inference time is " + str(per_sample)+ ' ms')
#     return per_sample, accuracy

In [9]:
def get_cpu_inference_time_and_test_acc(model, test_loader):
    test_data_size = len(testLoader.dataset)
    cpu_device = torch.device("cpu")
    model = model.to(cpu_device)
    # Wam-up
    # Because I sequentially run each cell above, and they includes for loop, I assume the CPU is warmed up
#     torch.cuda.synchronize()
    elapsed_time_ms = 0
    start = timer()
    # Start Inference Phase
    correct = 0
    with torch.no_grad():
        for imgs, labels in tqdm.tqdm(test_loader):
#             imgs, labels = imgs.cuda(), labels.cuda()
            imgs, labels = imgs.to(cpu_device), labels.to(cpu_device)
            out = model(imgs.float())
            predictions = out.argmax(dim=1, keepdim=True)  
            correct += predictions.eq(labels.view_as(predictions)).sum().item()
#             break
    accuracy = correct/len(test_loader.dataset)
    print(accuracy)
    # End of inference Phase
#     torch.cuda.synchronize()
    end = timer()
    elapsed_time_ms = (end - start) * 1000
    per_sample = elapsed_time_ms/test_data_size
    print("CPU Inference time is " + str(per_sample)+ ' ms')
    return per_sample, accuracy

In [10]:
torch.cuda.empty_cache()
model.eval()
cpu_inf_time, acc = get_cpu_inference_time_and_test_acc(model, testLoader)

100%|██████████| 417/417 [02:04<00:00,  3.35it/s]

0.9122
CPU Inference time is 12.453586680299999 ms


In [11]:
size_vgg_bef =os.path.getsize('./vgg16_2.pth')
print('model size: {:.3f}MB'.format(size_vgg_bef/1024**2))

model size: 527.803MB


# Gonna modify the vgg net

In [12]:
def get_cpu_inference_time_and_test_acc(model, test_loader):
    test_data_size = len(testLoader.dataset)
    cpu_device = torch.device("cpu")
    model = model.to(cpu_device)
    # Wam-up
    # Because I sequentially run each cell above, and they includes for loop, I assume the CPU is warmed up
#     torch.cuda.synchronize()
    elapsed_time_ms = 0
    start = timer()
    # Start Inference Phase
    correct = 0
    with torch.no_grad():
        for imgs, labels in tqdm.tqdm(test_loader):
#             imgs, labels = imgs.cuda(), labels.cuda()
            imgs, labels = imgs.to(cpu_device), labels.to(cpu_device)
            out = model(imgs.float())
            predictions = out.argmax(dim=1, keepdim=True)  
            correct += predictions.eq(labels.view_as(predictions)).sum().item()
#             break
    accuracy = correct/len(test_loader.dataset)
    print(accuracy)
    # End of inference Phase
#     torch.cuda.synchronize()
    end = timer()
    elapsed_time_ms = (end - start) * 1000
    per_sample = elapsed_time_ms/test_data_size
    print("CPU Inference time is " + str(per_sample)+ ' ms')
    return per_sample, accuracy

In [13]:
# def get_gpu_inference_time_and_test_acc(model, test_loader):
#     test_data_size = len(testLoader.dataset)
#     model = model.cuda()
#     # Wam-up
#     # Because I sequentially run each cell above, and they includes for loop, I assume the CPU is warmed up
#     torch.cuda.synchronize()
#     elapsed_time_ms = 0
#     start = timer()
#     # Start Inference Phase
#     correct = 0
#     with torch.no_grad():
#         for imgs, labels in tqdm.tqdm(test_loader):
#             imgs, labels = imgs.cuda(), labels.cuda()
#             # imgs, labels = imgs, labels
#             out = model(imgs)
#             predictions = out.argmax(dim=1, keepdim=True)  
#             correct += predictions.eq(labels.view_as(predictions)).sum().item()
#     accuracy = correct/len(test_loader.dataset)
#     print(accuracy)
#     # End of inference Phase
#     torch.cuda.synchronize()
#     end = timer()
#     elapsed_time_ms = (end - start) * 1000
#     per_sample = elapsed_time_ms/test_data_size
#     print("GPU Inference time is " + str(per_sample)+ ' ms')
#     return per_sample, accuracy

## For static quanlitization

In [14]:
import torch
import torchvision.models as models
from torch.ao.quantization import get_default_qconfig, prepare, convert, fuse_modules
from torch.ao.quantization import QConfig, MinMaxObserver, HistogramObserver

In [15]:
cpu_device = torch.device("cpu")

model_to_quantize = copy.deepcopy(model).to(cpu_device)
model_to_quantize

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [16]:
from torch.quantization import QuantStub, DeQuantStub

class QuantizedVGG(nn.Module):
    def __init__(self, model):
        super(QuantizedVGG, self).__init__()
        self.quant = QuantStub()
        self.vgg = model
        self.dequant = DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = self.vgg(x)
        return self.dequant(x)

model_reload = vgg16(pretrained=True)
state_dict = torch.load('./vgg16_2.pth',map_location=torch.device('cpu'))
model_reload.load_state_dict(state_dict)
model_reload = QuantizedVGG(model)


In [17]:
# gpu_device = torch.device("cuda:0")

# model_to_quantize = copy.deepcopy(model).to(gpu_device)
model_to_quantize = copy.deepcopy(model_reload)

# let the model be the evaluation mode
model_to_quantize.eval()

# obtain the qualization configeration
# qconfig = QConfig(activation=HistogramObserver.with_args(reduce_range=True),
#                   weight=MinMaxObserver.with_args(dtype=torch.qint8, qscheme=torch.per_tensor_affine))

# deploy at ARM
qconfig = get_default_qconfig("qnnpack")

# prepare the model, insert a observer
model_to_quantize.qconfig = qconfig
model_to_quantize = prepare(model_to_quantize)

#  torch.cuda.empty_cache()  # release gpu sources

# using dataset to correct
with tqdm.tqdm(total=len(trainLoader)) as pbar:
    pbar.set_description("Processing")
    for data, _ in trainLoader:
#         data = data.to(gpu_device)
        model_to_quantize(data)
        pbar.update(1)
#         break
        
# convert the model
model_to_quantize_final = convert(model_to_quantize)

Processing: 100%|██████████| 2084/2084 [13:55<00:00,  2.49it/s]


In [18]:
cpu_device = torch.device("cpu")

model_to_quantize_final = model_to_quantize_final.to(cpu_device)
cpu_inf_time, acc = get_cpu_inference_time_and_test_acc(model_to_quantize_final, testLoader)

100%|██████████| 417/417 [00:54<00:00,  7.67it/s]

0.9099
CPU Inference time is 5.436205164700004 ms


In [19]:
torch.save(model_to_quantize_final.state_dict(), './vgg16_2_aft_sta_qua.pth')
size_vgg_aft = os.path.getsize('./vgg16_2_aft_sta_qua.pth')
print('model size: {:.3f}MB'.format(size_vgg_aft/1024**2))

model size: 132.008MB


# For dynamic quantilization

In [20]:
cpu_device = torch.device("cpu")

model_to_dy_quantize = copy.deepcopy(model).to(cpu_device)
# model_to_dy_quantize

In [21]:
model_to_dy_quantize

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [22]:
# the layer which can be dynamic qconfig
# qconfig_spec = {
#                 nn.Linear : default_dynamic_qconfig,
#                 nn.LSTM : default_dynamic_qconfig,
#                 nn.GRU : default_dynamic_qconfig,
#                 nn.LSTMCell : default_dynamic_qconfig,
#                 nn.RNNCell : default_dynamic_qconfig,
#                 nn.GRUCell : default_dynamic_qconfig,
#             }

In [23]:
# create a quantized model instance
model_int8 = torch.quantization.quantize_dynamic(
    model_to_dy_quantize,  # the original model
    {torch.nn.Sequential, torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8)  # the target dtype for quantized weights
model_int8.to(cpu_device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [24]:
cpu_device = torch.device("cpu")

model_int8 = model_int8.to(cpu_device)
cpu_inf_time, acc = get_cpu_inference_time_and_test_acc(model_int8, testLoader)

100%|██████████| 417/417 [01:33<00:00,  4.46it/s]

0.9125
CPU Inference time is 9.345385786099996 ms


In [25]:
torch.save(model_int8.state_dict(), './vgg16_2_aft_dyna_qua.pth')
size_vgg_aft = os.path.getsize('./vgg16_2_aft_dyna_qua.pth')
print('model size: {:.3f}MB'.format(size_vgg_aft/1024**2))

model size: 174.086MB
